In this python notebooks, we train 3 models as per the paper.

1. Bag Of tricks
2. CNN Model
3. CNN 3 layer architecture.

These models are used for 'regular' ICM codes as outputs. 

More details can be found in the paper.

Attach Google Drive

In [1]:
from google.colab import drive
import os
drive.mount("/content/drive/")

mypath = "drive/My Drive/CS598DLHProject"
os.listdir(mypath)

Mounted at /content/drive/


['data',
 'BOTKeras.ipynb',
 'CNNModel.ipynb',
 'CNN3Layer.ipynb',
 'Reference',
 'storedModels',
 '.ipynb_checkpoints',
 'Preprocess.ipynb',
 'BOTModel.ipynb',
 'RegularYModelReference.ipynb',
 'BOTModelReference.ipynb',
 'OriginalModelRegularData.ipynb',
 'OriginalModelRolledData.ipynb']

Unzip the data



In [ ]:
# Already unzipped. Not required

# !unzip drive/My\ Drive/CS598DLHProject/data/original_paper_data/data.npz.zip -d drive/My\ Drive/CS598DLHProject/data/original_paper_data/

Archive:  drive/My Drive/CS598DLHProject/data/original_paper_data/data.npz.zip
  inflating: drive/My Drive/CS598DLHProject/data/original_paper_data/data.npz  
  inflating: drive/My Drive/CS598DLHProject/data/original_paper_data/__MACOSX/._data.npz  


Convert the model into relevant attributes. (X, Y_rolled, Y_regular and categories.)

In [ ]:
# Used to convert input data data.npz into x,cats,rolled,regular but not required now

# import gzip
# import pickle as pkl
# import gc

# import numpy as np

# DATA_DIR = mypath + "/data/"

# DATA_PATH = DATA_DIR + "preprocessed_3004/"
# DATA_NPZ_FILE = f'{DATA_PATH}data.npz'

# print ('reading from saved file Data.npz: ', DATA_NPZ_FILE)
# data = np.load(DATA_NPZ_FILE)
# lst = data.files
# for item in lst:
#     print(item)
#     # print(data[item])

# print("done")
# x = data['x']
# cats = data['cats']
# y_rolled = data['rol_y']
# y_full = data['reg_y']

# print(x.shape)
# print(cats.shape)
# print(y_rolled.shape)
# print(y_full.shape)

reading from saved file Data.npz:  drive/My Drive/CS598DLHProject/data/preprocessed_3004/data.npz
x
cats
reg_y
rol_y
done
(399623, 2199)
(399623, 15)
(399623, 781)
(399623, 4103)


Distribute the data into Train/Test/Validation data and save the data



In [ ]:
# Temporary block to convert data.npz into train/val/test

# texts = x
# texts_categories = cats
# regular_labels = y_full
# rolled_labels = y_rolled

# # Split
# s1 = int(.64 * len(texts))
# s2 = int(.8 * len(texts))
# x_train, x_val, x_test = np.split(texts, [s1, s2])
# cats_train, cats_val, cats_test = np.split(texts_categories, [s1, s2])
# reg_y_train, reg_y_val, reg_y_test = np.split(regular_labels, [s1, s2])
# rol_y_train, rol_y_val, rol_y_test = np.split(rolled_labels, [s1, s2])

# np.savez(f'{DATA_PATH}dataTrainValTest.npz',
#           x_train=x_train, x_val=x_val, x_test=x_test,
#           cats_train=cats_train, cats_val=cats_val, cats_test=cats_test,
#           reg_y_train=reg_y_train, reg_y_val=reg_y_val, reg_y_test=reg_y_test,
#           rol_y_train=rol_y_train, rol_y_val=rol_y_val, rol_y_test=rol_y_test)

Pull in data for training

In [2]:
import gzip
import pickle as pkl
import gc

import numpy as np

DATA_DIR = mypath + "/data/"

DATA_PATH = DATA_DIR + "preprocessed_3004/"
ORIGINAL_MODELS = DATA_PATH + "originalModels/"

DATA_NPZ_FILE = f'{DATA_PATH}dataTrainValTest.npz'

print ('reading from saved file dataTrainValTest.npz: ', DATA_NPZ_FILE)
data = np.load(DATA_NPZ_FILE)
lst = data.files
for item in lst:
    print(item)
    # print(data[item])

print("done")
x_train_load = data['x_train']
x_val_load = data['x_val']
# x_test = data['x_test']

ratio = 1
x_train = x_train_load[:int(ratio*x_train_load.shape[0])]
x_val = x_val_load[:int(ratio*x_val_load.shape[0])]

del x_train_load
del x_val_load

print('x_train: ' , x_train.shape)
print('x_val: ', x_val.shape)

rol_y_train = data['rol_y_train']
rol_y_val = data['rol_y_val']

print('rol_y_train: ', rol_y_train.shape)
print('rol_y_val: ', rol_y_val.shape)

y_train = rol_y_train[:int(ratio*rol_y_train.shape[0])]
y_val = rol_y_val[:int(ratio*rol_y_val.shape[0])]

print('y_train: ', y_train.shape)
print('y_val: ', y_val.shape)

del data
del lst
del rol_y_train
del rol_y_val

gc.collect()


reading from saved file dataTrainValTest.npz:  drive/My Drive/CS598DLHProject/data/preprocessed_3004/dataTrainValTest.npz
x_train
x_val
x_test
cats_train
cats_val
cats_test
reg_y_train
reg_y_val
reg_y_test
rol_y_train
rol_y_val
rol_y_test
done
x_train:  (255758, 2199)
x_val:  (63940, 2199)
rol_y_train:  (255758, 781)
rol_y_val:  (63940, 781)
y_train:  (255758, 781)
y_val:  (63940, 781)


43

In [3]:
with open(DATA_PATH + 'row_index_dictionary.p', 'rb') as f:
    word_index = pkl.load(f)
    print("Got word index")

Got word index


In [4]:
embedding_matrix = np.load(DATA_PATH + 'embedding_matrix.p', allow_pickle=True)

In [5]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Conv1D
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
import keras
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(len(word_index) , embedding_matrix.shape[1],
                    weights=[embedding_matrix], input_length=x_train.shape[1],
                    trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])
early_stopping = EarlyStopping(min_delta=.0001, patience=2)

filepath= ORIGINAL_MODELS + "BOTModelOriginal-{epoch:02d}-{loss:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

model.fit(x_train, y_train,
          batch_size=32, epochs=25,
          validation_data=[x_val, y_val],
          callbacks=[checkpoint])
# model.save('{}.h5'.format(filename))

Epoch 1/25
7982/7993 [============================>.] - ETA: 0s - loss: 0.0621 - precision_2: 0.6628 - recall_2: 0.1135
Epoch 1: saving model to drive/My Drive/CS598DLHProject/data/preprocessed_3004/originalModels/BOTModelOriginal-01-0.06.h5
7993/7993 [==============================] - 30s 4ms/step - loss: 0.0621 - precision_2: 0.6629 - recall_2: 0.1135 - val_loss: 0.0608 - val_precision_2: 0.9074 - val_recall_2: 0.0807
Epoch 2/25
7982/7993 [============================>.] - ETA: 0s - loss: 0.0608 - precision_2: 0.7128 - recall_2: 0.1164
Epoch 2: saving model to drive/My Drive/CS598DLHProject/data/preprocessed_3004/originalModels/BOTModelOriginal-02-0.06.h5
7993/7993 [==============================] - 29s 4ms/step - loss: 0.0608 - precision_2: 0.7126 - recall_2: 0.1165 - val_loss: 0.0606 - val_precision_2: 0.7054 - val_recall_2: 0.1203
Epoch 3/25
7993/7993 [==============================] - ETA: 0s - loss: 0.0606 - precision_2: 0.7133 - recall_2: 0.1202
Epoch 3: saving model to drive/M

In [9]:
# model.save(ORIGINAL_MODELS + 'BOTModelOriginal.h5')

In [10]:
import keras
from keras.callbacks import ModelCheckpoint


model = Sequential()
model.add(Embedding(len(word_index) , embedding_matrix.shape[1],
                    weights=[embedding_matrix], input_length=x_train.shape[1],
                    trainable=False))
model.add(Conv1D(250, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])


early_stopping = EarlyStopping(min_delta=.0001, patience=10)

filepath= ORIGINAL_MODELS + "CNNModelOriginal-{epoch:02d}-{loss:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

model.fit(x_train, y_train,
          batch_size=32, epochs=25,
          validation_data=[x_val, y_val],
          callbacks=[early_stopping, checkpoint])

Epoch 1/25
7993/7993 [==============================] - ETA: 0s - loss: 0.0595 - precision_3: 0.6994 - recall_3: 0.1480
Epoch 1: saving model to drive/My Drive/CS598DLHProject/data/preprocessed_3004/originalModels/CNNModelOriginal-01-0.06.h5
7993/7993 [==============================] - 153s 18ms/step - loss: 0.0595 - precision_3: 0.6994 - recall_3: 0.1480 - val_loss: 0.0573 - val_precision_3: 0.7721 - val_recall_3: 0.1612
Epoch 2/25
7992/7993 [============================>.] - ETA: 0s - loss: 0.0564 - precision_3: 0.7532 - recall_3: 0.1840
Epoch 2: saving model to drive/My Drive/CS598DLHProject/data/preprocessed_3004/originalModels/CNNModelOriginal-02-0.06.h5
7993/7993 [==============================] - 150s 19ms/step - loss: 0.0564 - precision_3: 0.7532 - recall_3: 0.1840 - val_loss: 0.0557 - val_precision_3: 0.7626 - val_recall_3: 0.1893
Epoch 3/25
7991/7993 [============================>.] - ETA: 0s - loss: 0.0551 - precision_3: 0.7570 - recall_3: 0.2022
Epoch 3: saving model to dri

In [11]:
# model.save(ORIGINAL_MODELS + 'CNNBaselineOriginal.h5')

In [12]:
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Conv1D, GlobalMaxPooling1D, concatenate
from keras.callbacks import ModelCheckpoint


embedding_input = Input(shape=(x_train.shape[1],), dtype=np.int32)
embedding_layer = Embedding(len(word_index), embedding_matrix.shape[1],
                                weights=[embedding_matrix], input_length=x_train.shape[1],
                                trainable=False)(embedding_input)
x1 = Conv1D(250, 2, activation='relu')(embedding_layer)
x1 = GlobalMaxPooling1D()(x1)
x2 = Conv1D(250, 3, activation='relu')(embedding_layer)
x2 = GlobalMaxPooling1D()(x2)
x3 = Conv1D(250, 4, activation='relu')(embedding_layer)
x3 = GlobalMaxPooling1D()(x3)
x = concatenate([x1, x2, x3])
output = Dense(y_train.shape[1], activation='sigmoid')(x)

model = Model(inputs=[embedding_input], outputs=[output])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

filepath= ORIGINAL_MODELS + "CNN3LayerModelOriginal-{epoch:02d}-{loss:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

early_stopping = EarlyStopping(min_delta=.0001, patience=2)
model.fit(x_train, y_train,
          batch_size=32, epochs=25,
          validation_data=[x_val, y_val],
          callbacks=[early_stopping, checkpoint])

Epoch 1/25
7993/7993 [==============================] - ETA: 0s - loss: 0.0605 - precision_4: 0.6375 - recall_4: 0.2695
Epoch 1: saving model to drive/My Drive/CS598DLHProject/data/preprocessed_3004/originalModels/CNN3LayerModelOriginal-01-0.06.h5
7993/7993 [==============================] - 452s 56ms/step - loss: 0.0605 - precision_4: 0.6375 - recall_4: 0.2695 - val_loss: 0.0546 - val_precision_4: 0.7163 - val_recall_4: 0.2766
Epoch 2/25
7992/7993 [============================>.] - ETA: 0s - loss: 0.0524 - precision_4: 0.7063 - recall_4: 0.3123
Epoch 2: saving model to drive/My Drive/CS598DLHProject/data/preprocessed_3004/originalModels/CNN3LayerModelOriginal-02-0.05.h5
7993/7993 [==============================] - 449s 56ms/step - loss: 0.0524 - precision_4: 0.7063 - recall_4: 0.3123 - val_loss: 0.0508 - val_precision_4: 0.7284 - val_recall_4: 0.3159
Epoch 3/25
7992/7993 [============================>.] - ETA: 0s - loss: 0.0493 - precision_4: 0.7333 - recall_4: 0.3322
Epoch 3: saving 

In [13]:
# model.save(ORIGINAL_MODELS + 'CNN3LayerOriginal.h5')